In [2]:
import os

In [3]:
%pwd

'd:\\End-to-End ML project\\NLP\\End-to-End projects\\Text-Summarization-NLP-project\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'd:\\End-to-End ML project\\NLP\\End-to-End projects\\Text-Summarization-NLP-project'

In [6]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelTrainerConfig:
  root_dir: Path
  data_path: Path
  model_ckpt: Path
  num_train_epochs: int
  warmup_steps: int
  per_device_train_batch_size: int
  weight_decay: float
  logging_steps: int
  evaluation_strategy: str
  eval_steps: int
  save_steps: float
  gradient_accumulation_steps: int

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath= PARAMS_FILE_PATH):

      self.config = read_yaml(config_filepath)
      self.params = read_yaml(params_filepath)

      create_directories([self.config.artifacts_root])


    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.evaluation_strategy,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

        return model_trainer_config

In [9]:
from transformers import TrainingArguments
from transformers import DataCollatorForSeq2Seq,AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

C:\Users\Dell\AppData\Roaming\Python\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-07-20 08:19:08,714: INFO: config: PyTorch version 2.7.1 available.]
[2025-07-20 08:19:08,723: INFO: config: TensorFlow version 2.10.0 available.]


In [10]:
from transformers import Trainer 

In [11]:
class ModelTrainer:
  def __init__(self,config:ModelTrainerConfig):
    self.config=config 

  def train_the_model(self):
    device= "cuda" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
    model_t5 = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
    seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_t5)

    #loading data 
    dataset_samsum_pt = load_from_disk(self.config.data_path)

    #use yaml file to load parameters or hardcode it in TrainingArg function 
    # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epochs, warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
        #     weight_decay=self.config.weight_decay, logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        # )

    #hardcode it in this function
    trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=0,
            per_device_train_batch_size=4, per_device_eval_batch_size=4,
            weight_decay=0.01, logging_steps=50,
            evaluation_strategy='epoch', save_strategy='no',disable_tqdm=True,
            gradient_accumulation_steps=1,report_to=[],overwrite_output_dir=True
        ) 
    trainer=Trainer(model=model_t5,args=trainer_args,tokenizer=tokenizer,
                    data_collator=seq2seq_data_collator,
                    train_dataset=dataset_samsum_pt['train'].select(range(1000)),
                    eval_dataset=dataset_samsum_pt['validation'])
    
    trainer.train()

    ##save model
    model_t5.save_pretrained(os.path.join(self.config.root_dir,"t5-samsum-model"))

    ##save tokenizer
    tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))



In [12]:
try:
  config=ConfigurationManager()
  model_trainer_config=config.get_model_trainer_config()
  model_trainer_config=ModelTrainer(config=model_trainer_config)
  model_trainer_config.train_the_model()
except Exception as e:
  raise e

[2025-07-20 08:19:28,013: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-20 08:19:28,023: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-20 08:19:28,026: INFO: common: created directory at:artifacts]
[2025-07-20 08:19:28,029: INFO: common: created directory at:artifacts/model_trainer]


d:\End-to-End ML project\venv\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
C:\Users\Dell\AppData\Roaming\Python\Python39\site-packages\networkx\utils\backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))
d:\End-to-End ML project\venv\lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'loss': 2.7237, 'grad_norm': 6.861265659332275, 'learning_rate': 4e-05, 'epoch': 0.2}
{'loss': 2.351, 'grad_norm': 4.0995192527771, 'learning_rate': 3e-05, 'epoch': 0.4}
{'loss': 2.3384, 'grad_norm': 4.243740558624268, 'learning_rate': 2e-05, 'epoch': 0.6}
{'loss': 2.3462, 'grad_norm': 3.936918258666992, 'learning_rate': 1e-05, 'epoch': 0.8}
{'loss': 2.3241, 'grad_norm': 5.630183219909668, 'learning_rate': 0.0, 'epoch': 1.0}
{'eval_loss': 2.0395901203155518, 'eval_runtime': 439.0722, 'eval_samples_per_second': 1.863, 'eval_steps_per_second': 0.467, 'epoch': 1.0}
{'train_runtime': 2732.5835, 'train_samples_per_second': 0.366, 'train_steps_per_second': 0.091, 'train_loss': 2.416666534423828, 'epoch': 1.0}
